# F1 Dashbord

### Goal
- creating a interactive dashborad showcasing different viszalizations about F1


### Data
- Data is available as an MySQL dump or in CSV files (I will use the MySQL database to train my sql skills)
- [Ergast Devloper API](http://ergast.com/mrd)

## Imports

In [1]:
import mysql.connector
import pandas as pd

## Explore MySQL Databse

The devloper of the API provied a Schema and User Guide so I don't need to explore the database in detail in this notebook: 
- [Schema](http://ergast.com/images/ergast_db.png)
- [User Guide](http://ergast.com/docs/f1db_user_guide.txt)

List of Tables:
- `circuits`
- `constructorResults`
- `constructorStandings`
- `constructors`
- `driverStandings`
- `drivers`
- `lapTimes`
- `pitStops`
- `qualifying`
- `races`
- `results`
- `seasons`
- `status`


In [5]:
# Establish connection
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "password",
    database = "f1db" 
)

# Create cursor
mycursor = mydb.cursor()

mycursor.execute("""
        SELECT *
        FROM circuits;
""")

result = mycursor.fetchall()

for x in result:
    print(x)


(1, 'albert_park', 'Albert Park Grand Prix Circuit', 'Melbourne', 'Australia', -37.8497, 144.968, 10, 'http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit')
(2, 'sepang', 'Sepang International Circuit', 'Kuala Lumpur', 'Malaysia', 2.76083, 101.738, 18, 'http://en.wikipedia.org/wiki/Sepang_International_Circuit')
(3, 'bahrain', 'Bahrain International Circuit', 'Sakhir', 'Bahrain', 26.0325, 50.5106, 7, 'http://en.wikipedia.org/wiki/Bahrain_International_Circuit')
(4, 'catalunya', 'Circuit de Barcelona-Catalunya', 'Montmeló', 'Spain', 41.57, 2.26111, 109, 'http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya')
(5, 'istanbul', 'Istanbul Park', 'Istanbul', 'Turkey', 40.9517, 29.405, 130, 'http://en.wikipedia.org/wiki/Istanbul_Park')
(6, 'monaco', 'Circuit de Monaco', 'Monte-Carlo', 'Monaco', 43.7347, 7.42056, 7, 'http://en.wikipedia.org/wiki/Circuit_de_Monaco')
(7, 'villeneuve', 'Circuit Gilles Villeneuve', 'Montreal', 'Canada', 45.5, -73.5228, 13, 'http://en.wikipedia.org/wiki

In [15]:
# Helper function
def sqlQuery(query):
    """
    returns a pandas DataFrame
    """
    # Create a cursor object
    mycursor = mydb.cursor()
    
    # Submit query
    mycursor.execute(query)

    # Call query
    result = mycursor.fetchall()
    
    # Column names from query
    col_names = mycursor.column_names

    # Get rows
    rows_lst = []
    for row in result:
        rows_lst.append(row)

    # Create DataFrame
    df = pd.DataFrame(columns=col_names, data=rows_lst)

    return df

### Statistics About the Drivers

In [6]:
mycursor = mydb.cursor()

mycursor.execute("""
        SELECT *
        FROM drivers;
""")

result = mycursor.fetchall()

# get column names from table
column_names = mycursor.column_names

row_list = []
for x in result:
    row_list.append(x)

df = pd.DataFrame(columns=column_names, data=row_list)

In [7]:
df

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
849,851,aitken,89.0,AIT,Jack,Aitken,1995-09-23,British,http://en.wikipedia.org/wiki/Jack_Aitken
850,852,tsunoda,22.0,TSU,Yuki,Tsunoda,2000-05-11,Japanese,http://en.wikipedia.org/wiki/Yuki_Tsunoda
851,853,mazepin,9.0,MAZ,Nikita,Mazepin,1999-03-02,Russian,http://en.wikipedia.org/wiki/Nikita_Mazepin
852,854,mick_schumacher,47.0,MSC,Mick,Schumacher,1999-03-22,German,http://en.wikipedia.org/wiki/Mick_Schumacher


#### Nationality of the drivers

In [23]:
query = ("""
        SELECT nationality, COUNT(1) AS number
        FROM drivers
        GROUP BY nationality
        ORDER BY number DESC;
""")

# using helper function to get data
df_nationality = sqlQuery(query)

df_nationality


,nationality,number
0,British,165
1,American,157
2,Italian,99
3,French,73
4,German,50
5,Brazilian,32
6,Argentine,24
7,South African,23
8,Swiss,23
9,Belgian,23


##### Questions: Why so many American F1 drivers?

In [29]:
query = ("""
    SELECT *
    FROM drivers
    WHERE nationality = "American" AND code IS NOT NULL;
""")

df_nationality_american = sqlQuery(query)

df_nationality_american

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,26,speed,NaN,SPE,Scott,Speed,1983-01-24,American,http://en.wikipedia.org/wiki/Scott_Speed
1,834,rossi,53.0,RSS,Alexander,Rossi,1991-09-25,American,http://en.wikipedia.org/wiki/Alexander_Rossi_%...


In [30]:
query = ("""
    SELECT *
    FROM drivers
    WHERE nationality = "Mexican";
""")

df_nationality_american = sqlQuery(query)

df_nationality_american

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,212,rebaque,NaN,None,Hector,Rebaque,1956-02-05,Mexican,http://en.wikipedia.org/wiki/Hector_Rebaque
1,345,rodriguez,NaN,None,Pedro,Rodríguez,1940-01-18,Mexican,http://en.wikipedia.org/wiki/Pedro_Rodr%C3%ADg...
2,382,solana,NaN,None,Moisés,Solana,1935-12-26,Mexican,http://en.wikipedia.org/wiki/Mois%C3%A9s_Solana
3,454,ricardo_rodriguez,NaN,None,Ricardo,Rodríguez,1942-02-14,Mexican,http://en.wikipedia.org/wiki/Ricardo_Rodr%C3%A...
4,815,perez,11.0,PER,Sergio,Pérez,1990-01-26,Mexican,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez
5,821,gutierrez,21.0,GUT,Esteban,Gutiérrez,1991-08-05,Mexican,http://en.wikipedia.org/wiki/Esteban_Guti%C3%A...


In [33]:
query = ("""
    SELECT *
    FROM drivers
    WHERE forename = "Esteban";
""")

df_nationality_american = sqlQuery(query)

df_nationality_american

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,74,tuero,NaN,None,Esteban,Tuero,1978-04-22,Argentine,http://en.wikipedia.org/wiki/Esteban_Tuero
1,821,gutierrez,21.0,GUT,Esteban,Gutiérrez,1991-08-05,Mexican,http://en.wikipedia.org/wiki/Esteban_Guti%C3%A...
2,839,ocon,31.0,OCO,Esteban,Ocon,1996-09-17,French,http://en.wikipedia.org/wiki/Esteban_Ocon


#### Driver per year and standing

- join `drivers`, `qualifying` and `races`

In [54]:
query = ("""
    SELECT DISTINCT(d.driverId), d.forename, d.surname, r.year
    FROM drivers AS d
    JOIN qualifying AS q ON d.driverId = q.driverId
    JOIN races AS r ON q.raceId = r.raceId
    WHERE d.surname = "Schumacher"
    ORDER BY year DESC;
""")

df_driver_year = sqlQuery(query)
df_driver_year

,driverId,forename,surname,year
0,854,Mick,Schumacher,2022
1,854,Mick,Schumacher,2021
2,30,Michael,Schumacher,2012
3,30,Michael,Schumacher,2011
4,30,Michael,Schumacher,2010
5,23,Ralf,Schumacher,2007
6,30,Michael,Schumacher,2006
7,23,Ralf,Schumacher,2006
8,23,Ralf,Schumacher,2005
9,30,Michael,Schumacher,2005


In [61]:
query = ("""
    SELECT d.driverId, s.raceId, d.forename, d.surname,  s.points, s.position
    FROM driverStandings AS s
    JOIN drivers AS d ON d.driverId = s.driverId
    WHERE forename = "Lewis";
""")

df_constructorResults = sqlQuery(query)
df_constructorResults

,driverId,raceId,forename,surname,points,position
0,1,18,Lewis,Hamilton,10.0,1
1,1,19,Lewis,Hamilton,14.0,1
2,1,20,Lewis,Hamilton,14.0,3
3,1,21,Lewis,Hamilton,20.0,2
4,1,22,Lewis,Hamilton,28.0,3
...,...,...,...,...,...,...
285,1,1071,Lewis,Hamilton,318.5,2
286,1,1072,Lewis,Hamilton,369.5,2
287,1,1073,Lewis,Hamilton,387.5,2
288,1,1074,Lewis,Hamilton,15.0,3


## Which constructor was the most successful?